In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from src.utils import tensorprod, dagger, bra_ket, expectation
from src.gates import CX, H, rx, ry, rz, cry, p

In [ ]:
I2 = np.eye(2)

In [ ]:
Zero = np.array([1, 0]).reshape(-1, 1)
One = np.array([0, 1]).reshape(-1, 1)

In [ ]:
def Ansatz(x):
    """ZZ-feature map ansatz for 4 qubits with pairwise entanglement."""
    init = tensorprod(Zero, Zero, Zero, Zero)
    l_1 = tensorprod(H, H, H, H)
    l_2 = tensorprod(p(2 * x[0]), p(2 * x[1]), p(2 * x[2]), p(2 * x[3]))
    l_3 = tensorprod(CX, CX)
    l_4 = tensorprod(I2, p(2 * (np.pi - x[0]) * (np.pi - x[1])), I2, p(2 * (np.pi - x[2]) * (np.pi - x[3])))
    l_5 = tensorprod(CX, CX)
    l_6 = tensorprod(I2, CX, I2)
    l_7 = tensorprod(I2, I2, p(2 * (np.pi - x[1]) * (np.pi - x[2])), I2)
    l_8 = tensorprod(I2, CX, I2)
    return l_8 @ l_7 @ l_6 @ l_5 @ l_4 @ l_3 @ l_2 @ l_1 @ init

In [ ]:
p(np.pi) @ p(np.pi)

In [ ]:
tensorprod(p(np.pi), p(np.pi)).shape

In [ ]:
from sklearn.datasets import make_classification
X, y = make_classification(1000, 4, random_state=137)

In [ ]:
X[0]

In [ ]:
%%timeit
Ansatz(X[0])

In [ ]:
np.conj(Ansatz(X[0])).reshape(-1) @ Ansatz(X[0])

In [ ]:
Ansatz(X[0])

# Qiskit

In [ ]:
from qiskit.quantum_info import Statevector
from qiskit.circuit.library import ZZFeatureMap

In [ ]:
zf = ZZFeatureMap(4, reps=1, entanglement='pairwise')

In [ ]:
zf.assign_parameters(dict(
    zip(zf.parameters, X[0][::-1])
), inplace=True)

In [ ]:
zf.decompose().draw()

In [ ]:
state = Statevector.from_int(0, 2**4).evolve(zf)

In [ ]:
state = np.array(state).reshape(-1, 1)

In [ ]:
np.isclose(state, Ansatz(X[0]))